In [2]:
# Import packages
from osgeo import gdal
import os
import glob

import matplotlib.pyplot as plt
#from mpl_toolkits.axes_grid1 import make_axes_locatabl
import numpy as np
import rasterio
import xarray
import rioxarray

In [110]:
# Define filepaths
filepath7 = '/Users/jsheppar/Downloads/landsat7/'
filepath8 = '/Users/jsheppar/Downloads/landsat8/'
filepath_DF_mask = '/Users/jsheppar/Documents/GitHub/Landslide_NDVI/DF_area1.tif.aux/DF_area1_32610.tif'
filepath_DF_ownership = '/Users/jsheppar/Documents/GitHub/Landslide_NDVI/Ownership_DF/Ownership_DF_32610.tif'

# Define list of Landsat bands
files7 = sorted(glob.glob(filepath7 + '*.tif'))
files8 = sorted(glob.glob(filepath8 + '*.tif'))

In [107]:
def process_ndvi_7(files):
    #strip name from filepath for later
    name = files.split("/")[-1]
    xds = rioxarray.open_rasterio(files)
    
    #select band 4 and 3 from landsat 7 tif
    band4 = xds.sel(band=1)
    band3 = xds.sel(band=2)
    #band2 = xds.sel(band=3)
    
    # Compute NDVI
    #p.seterr(divide='ignore', invalid='ignore')
    ndvi = np.divide((band4 - band3), (band4 + band3))
    print(ndvi)
    
    # clip to shape of Douglas Fire by making x and y coordinates match AND the mask = 1
    clipped_ndvi = ndvi.where((ndvi.x == df_mask.x) and (ndvi.y == df_mask.y) and (df_mask == 1))
    
    print(clipped_ndvi) # seems to have no data and also loses the x coordinate. y coordinate remains. Why?
    
    clipped_ndvi.rio.to_raster("/Users/jsheppar/Downloads/landsat_ndvi/ndvi2_"+name) # Unable to determine bounds from coordinates.
    
def process_ndvi_8(files):
    name = files.split("/")[-1]
    xds = rioxarray.open_rasterio(files)
    
    band5 = xds.sel(band=1)
    band4 = xds.sel(band=2)
    #band3 = xds.sel(band=3)
    
    # Compute NDVI
    #p.seterr(divide='ignore', invalid='ignore')
    ndvi = np.divide((band5 - band4), (band5 + band4))
    
    ndvi.rio.to_raster("/Users/jsheppar/Downloads/landsat_ndvi/ndvi_"+name)

In [108]:
df_mask = rioxarray.open_rasterio(filepath_DF_mask)
df_ownership = rioxarray.open_rasterio(filepath_DF_ownership)

process_ndvi_7(files7[0])
#process_ndvi_8(files8)

<xarray.DataArray (y: 3895, x: 2361)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 4.416e+05 4.417e+05 ... 4.652e+05 4.652e+05
  * y            (y) float64 4.76e+06 4.76e+06 4.76e+06 ... 4.721e+06 4.721e+06
    spatial_ref  int64 0
<xarray.DataArray (y: 3895, x: 0)>
array([], shape=(3895, 0), dtype=float32)
Coordinates:
  * x            (x) float64 
  * y            (y) float64 4.76e+06 4.76e+06 4.76e+06 ... 4.721e+06 4.721e+06
    spatial_ref  int64 0


NoDataInBounds: Unable to determine bounds from coordinates.

In [109]:
print(df_mask)

<xarray.DataArray (band: 1, y: 876, x: 570)>
[499320 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 4.458e+05 4.458e+05 ... 4.628e+05 4.629e+05
  * y            (y) float64 4.749e+06 4.749e+06 ... 4.722e+06 4.722e+06
    spatial_ref  int64 0
Attributes:
    RepresentationType:  ATHEMATIC
    _FillValue:          -3.4028234663852886e+38
    scale_factor:        1.0
    add_offset:          0.0
